In [63]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [2]:
df=spark.read.format('json')\
        .load("learning_spark_data/2015-summary.json")

In [3]:
df.count()

256

In [4]:
df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [6]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [15]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [26]:
#df.collect()

In [16]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [17]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [27]:
df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [30]:
df1=df.select('DEST_COUNTRY_NAME').distinct().cache() #메모리유지
df1.count()

132

In [31]:
#ROW class를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow=Row('hello',None,1,False)
myRow

<Row('hello', None, 1, False)>

In [32]:
#새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3=df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME==DEST_COUNTRY_NAME')) #expr sql 표현식을 받아 생성
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [33]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [35]:
df3.filter('withinCountry=True').show(3)

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [37]:
df3.where(df3.withinCountry == True).show(3)

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [ ]:
# case when 'count' 10이하는 under, 이상은 upper로 변환 > category컬럼 추가

In [39]:
from pyspark.sql.functions import when

In [45]:
df4=df3.withColumn('category', 
              when(df3['count']<=10, 'under').otherwise('upper'))
df4.select('count','category').show()

+-----+--------+
|count|category|
+-----+--------+
|   15|   upper|
|    1|   under|
|  344|   upper|
|   15|   upper|
|   62|   upper|
|    1|   under|
|   62|   upper|
|  588|   upper|
|   40|   upper|
|    1|   under|
|  325|   upper|
|   39|   upper|
|   64|   upper|
|    1|   under|
|   41|   upper|
|   30|   upper|
|    6|   under|
|    4|   under|
|  230|   upper|
|    1|   under|
+-----+--------+
only showing top 20 rows



In [47]:
df4.show(8)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
|    United States|          Singapore|    1|        false|   under|
|    United States|            Grenada|   62|        false|   upper|
|       Costa Rica|      United States|  588|        false|   upper|
+-----------------+-------------------+-----+-------------+--------+
only showing top 8 rows



In [ ]:
#DataFrame의 select(),where(),filter()트랜스포메이션
#show(), count()액션

In [58]:
spark.stop()

## 집계함수

In [50]:
# emp_df, dept_df

In [64]:
emp_df = spark.read.csv("learning_spark_data/emp.csv", header=True, inferSchema=True)
dept_df = spark.read.csv("learning_spark_data/dept.csv", header=True, inferSchema=True)

In [65]:
emp_df.show(5)

+-----+------+--------+----+----------+----+----+------+
|empno| ename|     job| mgr|  hiredate| sal|comm|deptno|
+-----+------+--------+----+----------+----+----+------+
| 7369| SMITH|   CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN|SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD|SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES| MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN|SALESMAN|7698|1981-09-28|1250|1400|    30|
+-----+------+--------+----+----------+----+----+------+
only showing top 5 rows



In [66]:
dept_df.show(5)

+------+----------+--------+
|deptno|     dname|     loc|
+------+----------+--------+
|    10|ACCOUNTING|NEW YORK|
|    20|  RESEARCH|  DALLAS|
|    30|     SALES| CHICAGO|
|    40|OPERATIONS|  BOSTON|
+------+----------+--------+



In [71]:
emp_df.select('ENAME','Deptno').show(5)

+------+------+
| ENAME|Deptno|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
+------+------+
only showing top 5 rows



In [68]:
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [69]:
#Expr을 통한 조건식
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [75]:
from pyspark.sql.functions import countDistinct
emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [76]:
from pyspark.sql.functions import approx_count_distinct #오차범위 설정
emp_df.select(approx_count_distinct('job',0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [ ]:
#first,last,min,max,sum,avg ->function으로 처리

In [155]:
emp_df.select(count('empno'), count('*'), count('comm'), max('ename'), max('comm')).show()

+------------+--------+-----------+----------+---------+
|count(empno)|count(1)|count(comm)|max(ename)|max(comm)|
+------------+--------+-----------+----------+---------+
|          15|      15|          4|      WARD|     1400|
+------------+--------+-----------+----------+---------+



In [82]:
from pyspark.sql.functions import first
emp_df.select(first('sal', ignorenulls=True)).show()

+----------+
|first(sal)|
+----------+
|       800|
+----------+



In [83]:
from pyspark.sql.functions import last
emp_df.select(last('sal',ignorenulls=True)).show()

+---------+
|last(sal)|
+---------+
|     3200|
+---------+



In [87]:
from pyspark.sql.functions import min
emp_df.select(min('sal')).show()

+--------+
|min(sal)|
+--------+
|     800|
+--------+



In [89]:
from pyspark.sql.functions import max
emp_df.select(max('sal')).show()

+--------+
|max(sal)|
+--------+
|    5000|
+--------+



In [153]:
from pyspark.sql.functions import col
emp_df.select(sum('sal')).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [97]:
emp_df.selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [154]:
from pyspark.sql.functions import avg
emp_df.select(avg('sal')).show()

+------------------+
|          avg(sal)|
+------------------+
|2148.3333333333335|
+------------------+



In [157]:
emp_df.select(sum('sal'), mean('sal'), avg('sal')).show()

+--------+------------------+------------------+
|sum(sal)|          avg(sal)|          avg(sal)|
+--------+------------------+------------------+
|   32225|2148.3333333333335|2148.3333333333335|
+--------+------------------+------------------+



In [ ]:
#total_sal/total_transaction, avg_salary,mean_salary

In [110]:
from pyspark.sql.functions import sum, count, avg, mean, round

emp_df.select(
    round(sum('sal') / count('sal'), 3).alias('total_sal_per_total_transaction'),
    round(avg('sal'), 3).alias('avg_salary'),
    round(mean('sal'), 3).alias('mean_salary')
).show()

+-------------------------------+----------+-----------+
|total_sal_per_total_transaction|avg_salary|mean_salary|
+-------------------------------+----------+-----------+
|                       2148.333|  2148.333|   2148.333|
+-------------------------------+----------+-----------+



In [108]:
#그룹화
emp_df.groupby('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [113]:
#select job,
#       count(job),
#       sum(sal)
#group by job
emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
).show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [115]:
#sal의 평균 SAL_AVG, 표준편차 SAL_STDEV를 job별로 계산해서 출력,소수점2자리
from pyspark.sql.functions import avg, stddev, round

emp_df.groupBy('job').agg(
    round(avg('sal'),2).alias('SAL_AVG'),
    round(stddev('sal'),2).alias('SAL_STDEV')
).show()


+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



### 급여 Top 10 구하기

In [158]:
emp_df.groupby('ename')\
    .agg(max('sal').alias('max_sal'))\
    .orderBy(col('max_sal').desc())\
.show(10)

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [159]:
emp_df.orderBy(emp_df.sal.desc()).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [124]:
#윈도우 함수 
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window

windowspec=Window.orderBy(desc('sal'))
salAllRank=rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [125]:
emp_df.show(10)

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
+-----+------+---------+----+----------+----+----+------+
only showing top 10 rows



In [126]:
emp_df.withColumn('salary rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [ ]:
# 직무별로 rank작성
#window.partitionBy()
#job_rank_df 작성

In [139]:
emp_df.withColumn('salary_rank_by_job', 
                  rank().over(Window.partitionBy('job').orderBy(desc('sal')))) \
    .select('ename','job', 'sal', 'salary_rank_by_job') \
    .show()

+------+---------+----+------------------+
| ename|      job| sal|salary_rank_by_job|
+------+---------+----+------------------+
| SCOTT|  ANALYST|3000|                 1|
|  FORD|  ANALYST|3000|                 1|
|  JACK|    CLERK|3200|                 1|
|MILLER|    CLERK|1300|                 2|
| ADAMS|    CLERK|1100|                 3|
| JAMES|    CLERK| 950|                 4|
| SMITH|    CLERK| 800|                 5|
| JONES|  MANAGER|2975|                 1|
| BLAKE|  MANAGER|2850|                 2|
| CLARK|  MANAGER|2450|                 3|
|  KING|PRESIDENT|5000|                 1|
| ALLEN| SALESMAN|1600|                 1|
|TURNER| SALESMAN|1500|                 2|
|  WARD| SALESMAN|1250|                 3|
|MARTIN| SALESMAN|1250|                 3|
+------+---------+----+------------------+



In [129]:
emp_df.dtypes

[('empno', 'int'),
 ('ename', 'string'),
 ('job', 'string'),
 ('mgr', 'int'),
 ('hiredate', 'date'),
 ('sal', 'int'),
 ('comm', 'int'),
 ('deptno', 'int')]

### 부서별 순위

In [162]:
from pyspark.sql.functions import rank, dense_rank, row_number, col, desc

#부서별 급여순위
dept_window_spec = Window.partitionBy('deptno').orderBy(desc('sal'))
df_with_dept_rank = emp_df.withColumn('dept_salary_rank', 
                                  row_number().over(dept_window_spec))
df_with_dept_rank.select('ename', 'deptno', 'sal', 'dept_salary_rank').show()

+------+------+----+----------------+
| ename|deptno| sal|dept_salary_rank|
+------+------+----+----------------+
|  KING|    10|5000|               1|
| CLARK|    10|2450|               2|
|MILLER|    10|1300|               3|
| SCOTT|    20|3000|               1|
|  FORD|    20|3000|               2|
| JONES|    20|2975|               3|
| ADAMS|    20|1100|               4|
| SMITH|    20| 800|               5|
| BLAKE|    30|2850|               1|
| ALLEN|    30|1600|               2|
|TURNER|    30|1500|               3|
|  WARD|    30|1250|               4|
|MARTIN|    30|1250|               5|
| JAMES|    30| 950|               6|
|  JACK|    70|3200|               1|
+------+------+----+----------------+



In [141]:
#누적 급여 sum('sal').over()


In [163]:
#부서별 누적급여1

window_spec_sum = Window.partitionBy('deptno').orderBy('empno')
df_cumulative = emp_df.withColumn('cumulative_salary', 
                                sum('sal').over(window_spec_sum))
df_cumulative.select('ename', 'deptno', 'sal', 'cumulative_salary').show()

+------+------+----+-----------------+
| ename|deptno| sal|cumulative_salary|
+------+------+----+-----------------+
| CLARK|    10|2450|             2450|
|  KING|    10|5000|             7450|
|MILLER|    10|1300|             8750|
| SMITH|    20| 800|              800|
| JONES|    20|2975|             3775|
| SCOTT|    20|3000|             6775|
| ADAMS|    20|1100|             7875|
|  FORD|    20|3000|            10875|
| ALLEN|    30|1600|             1600|
|  WARD|    30|1250|             2850|
|MARTIN|    30|1250|             4100|
| BLAKE|    30|2850|             6950|
|TURNER|    30|1500|             8450|
| JAMES|    30| 950|             9400|
|  JACK|    70|3200|             3200|
+------+------+----+-----------------+



In [150]:
#부서별 누적급여2

# 2. 조인
emp_dept_df = emp_df.join(
    dept_df,
    emp_df.deptno == dept_df.deptno,
    'inner'
)

# 3. 누적합
windowspec = Window.partitionBy('dname').orderBy('sal')
result_df = emp_dept_df.withColumn(
    'dept_cumsum_sal',
    sum('sal').over(windowspec)
)

# 4. 결과
result_df.select('dname', 'ename', 'sal', 'dept_cumsum_sal') \
    .orderBy('dname', 'sal') \
    .show()

+----------+------+----+---------------+
|     dname| ename| sal|dept_cumsum_sal|
+----------+------+----+---------------+
|ACCOUNTING|MILLER|1300|           1300|
|ACCOUNTING| CLARK|2450|           3750|
|ACCOUNTING|  KING|5000|           8750|
|  RESEARCH| SMITH| 800|            800|
|  RESEARCH| ADAMS|1100|           1900|
|  RESEARCH| JONES|2975|           4875|
|  RESEARCH| SCOTT|3000|          10875|
|  RESEARCH|  FORD|3000|          10875|
|     SALES| JAMES| 950|            950|
|     SALES|  WARD|1250|           3450|
|     SALES|MARTIN|1250|           3450|
|     SALES|TURNER|1500|           4950|
|     SALES| ALLEN|1600|           6550|
|     SALES| BLAKE|2850|           9400|
+----------+------+----+---------------+



In [164]:
#부서별 평균급여와 직원 개별 급여 비교1

window_spec_avg = Window.partitionBy('deptno')
df_avg_compare = emp_df.withColumn('dept_avg_salary', 
                        avg('sal').over(window_spec_avg))
df_avg_compare.select('ename', 'deptno', 'sal', 'dept_avg_salary').show()

+------+------+----+------------------+
| ename|deptno| sal|   dept_avg_salary|
+------+------+----+------------------+
| CLARK|    10|2450|2916.6666666666665|
|  KING|    10|5000|2916.6666666666665|
|MILLER|    10|1300|2916.6666666666665|
| SMITH|    20| 800|            2175.0|
| JONES|    20|2975|            2175.0|
| SCOTT|    20|3000|            2175.0|
| ADAMS|    20|1100|            2175.0|
|  FORD|    20|3000|            2175.0|
| ALLEN|    30|1600|1566.6666666666667|
|  WARD|    30|1250|1566.6666666666667|
|MARTIN|    30|1250|1566.6666666666667|
| BLAKE|    30|2850|1566.6666666666667|
|TURNER|    30|1500|1566.6666666666667|
| JAMES|    30| 950|1566.6666666666667|
|  JACK|    70|3200|            3200.0|
+------+------+----+------------------+



In [152]:
#부서별 평균급여와 직원 개별 급여 비교2

emp_dept_df=emp_df.join( dept_df,
                        emp_df.deptno==dept_df.deptno,
                        'inner'
                       )

#부서별 평균급여 구하기

windowspec=Window.partitionBy('dname')
emp_dept_with_avg=emp_dept_df.withColumn(
    'dept_avg_sal',
    avg('sal').over(windowspec)
)

#직원 급여와 부서 평균급여, 비교결과
result_df=emp_dept_with_avg.withColumn(
    'compare_to_avg',
    col('sal')-col('dept_avg_sal')
)

# 5. 결과 출력
result_df.select('dname', 'ename', 'sal', 'dept_avg_sal', 'compare_to_avg') \
    .orderBy('dname', 'ename') \
    .show()

+----------+------+----+------------------+-------------------+
|     dname| ename| sal|      dept_avg_sal|     compare_to_avg|
+----------+------+----+------------------+-------------------+
|ACCOUNTING| CLARK|2450|2916.6666666666665| -466.6666666666665|
|ACCOUNTING|  KING|5000|2916.6666666666665| 2083.3333333333335|
|ACCOUNTING|MILLER|1300|2916.6666666666665|-1616.6666666666665|
|  RESEARCH| ADAMS|1100|            2175.0|            -1075.0|
|  RESEARCH|  FORD|3000|            2175.0|              825.0|
|  RESEARCH| JONES|2975|            2175.0|              800.0|
|  RESEARCH| SCOTT|3000|            2175.0|              825.0|
|  RESEARCH| SMITH| 800|            2175.0|            -1375.0|
|     SALES| ALLEN|1600|1566.6666666666667|  33.33333333333326|
|     SALES| BLAKE|2850|1566.6666666666667| 1283.3333333333333|
|     SALES| JAMES| 950|1566.6666666666667| -616.6666666666667|
|     SALES|MARTIN|1250|1566.6666666666667|-316.66666666666674|
|     SALES|TURNER|1500|1566.66666666666

In [165]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [167]:
emp_dept_df=emp_df.join(dept_df,emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [168]:
join_df=emp_df.join(dept_df, on='deptno', how='inner')
join_df.select('ename','deptno','dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+



In [169]:
spark.stop()